In [ ]:
from prettytable import PrettyTable

def generate_three_address_code(code_lines):
    final_code = []
    temp_counter = 1
    do_index = None
    instruction_pointer = 100

    for line in code_lines:
        line = line.strip()
        if 'do' in line:
            do_index = instruction_pointer
            instruction_pointer += 1
            continue
        elif 'while' in line:
            conditions = line[line.find('(') + 1:line.rfind(')')].split('&&')
            for condition in conditions:
                final_code.append(f'if {condition.strip()} goto {do_index}')
                final_code.append('goto')
                instruction_pointer += 1
            final_code.append("End")
            instruction_pointer += 1
        else:
            statements = line.split(';')
            for statement in statements:
                if '=' in statement:
                    lhs, rhs = statement.split('=')
                    lhs = lhs.strip()
                    rhs = rhs.strip()
                    if '*' in rhs or '/' in rhs:
                        temp_var = f't{temp_counter}'
                        final_code.append(f'{temp_var} = {rhs}')
                        final_code.append(f'{lhs} = {temp_var}')
                        temp_counter += 1
                    elif '+' in rhs or '-' in rhs:
                        temp_var1 = f't{temp_counter}'
                        temp_counter += 1
                        temp_var2 = f't{temp_counter}'
                        final_code.append(f'{temp_var1} = {rhs}')
                        final_code.append(f'{temp_var2} = {temp_var1}')
                        final_code.append(f'{lhs} = {temp_var2}')
                        temp_counter += 1
                    elif rhs.isdigit():
                        final_code.append(f'{lhs} = {rhs}')
                    else:
                        temp_var = f't{temp_counter}'
                        final_code.append(f'{temp_var} = {rhs}')
                        final_code.append(f'{lhs} = {temp_var}')
                        temp_counter += 1
                    instruction_pointer += 1
                elif statement.strip():
                    final_code.append(statement.strip())
                    instruction_pointer += 1

    return final_code

def read_code_from_file(file_name):
    with open(file_name) as f:
        code_lines = f.readlines()
    return [line.strip() for line in code_lines if line.strip()]

def display_three_address_code(three_address_code):
    print('\nThe Three Address Code Generated is:')
    table = PrettyTable()
    table.align = "l"
    table.field_names = ['Index','Code']
    for i, code_line in enumerate(three_address_code):
        table.add_row([100 + i, code_line])
    print(table)

code_lines = read_code_from_file('code1.txt')
print('Statements:')
print(''.join(code_lines))
three_address_code = generate_three_address_code(code_lines)
display_three_address_code(three_address_code)


Statements:
do{a=b+c;b=t-7;c=x+z;}while(a<b && c<b);

The Three Address Code Generated is:
+-------+-----------------+
| Index | Code            |
+-------+-----------------+
| 100   | t1 = b+c        |
| 101   | t2 = t1         |
| 102   | a = t2          |
| 103   | t3 = t-7        |
| 104   | t4 = t3         |
| 105   | b = t4          |
| 106   | t5 = x+z        |
| 107   | t6 = t5         |
| 108   | c = t6          |
| 109   | if a<b goto 100 |
| 110   | goto            |
| 111   | if c<b goto 100 |
| 112   | goto            |
| 113   | End             |
+-------+-----------------+


In [ ]:
import numpy as np
import pandas as pd
from prettytable import PrettyTable

def dowhileloop(code):
    fc = []
    idx = None
    temp_counter = 1

    def new_temp():
        nonlocal temp_counter
        temp = f"t{temp_counter}"
        temp_counter += 1
        return temp

    for i in range(len(code)):
        line = code[i]

        if 'do' in line:
            idx = i
        elif 'while' in line:
            start_idx = line.index('(')
            end_idx = line.index(')')
            condition = line[start_idx+1:end_idx]

            if "and" in condition:
                conditions = condition.split('and')
                ln = len(fc)
                fc.append(f"if ({conditions[0].strip()}) goto {ln+3}")
                fc.append(f"goto {ln+4}")
                fc.append(f"if ({conditions[1].strip()}) goto {idx*2 + 1}")

            elif "or" in condition:
                conditions = condition.split('or')
                fc.append(f"if ({conditions[0].strip()}) goto {idx*2 + 1}")
                fc.append(f"if ({conditions[1].strip()}) goto {idx*2 + 1}")

        else:
            if '=' in line:
                parts = line.split('=')
                if len(parts) >= 2:  # Check if there are enough parts after splitting
                    var_name = parts[0].strip()
                    operation = parts[1].strip()

                    if '+' in operation or '-' in operation or '*' in operation or '/' in operation:
                        operands = operation.split()
                        if len(operands) >= 3:  # Check if there are enough operands after splitting
                            result_temp = new_temp()
                            fc.append(f"{result_temp} = {operands[0]} {operands[1]} {operands[2]}")
                            for j in range(3, len(operands), 2):
                                temp_var = new_temp()
                                fc.append(f"{temp_var} = {result_temp} {operands[j]} {operands[j+1]}")
                                result_temp = temp_var
                            fc.append(f"{var_name} = {result_temp}")
                        else:
                            fc.append(f"{var_name} = {operation}")  # Handle if there are not enough operands
                    else:
                        fc.append(f"{var_name} = {operation}")
                else:
                    fc.append(line)  # Handle if there are not enough parts after splitting
            else:
                fc.append(line)

    fc.append(f"END")

    return fc

with open('code1.txt') as f:
    code = f.readlines()

ans = []
for i in range(len(code)):
    if code[i] != '\n':
        if code[i][-1] == '\n':
            ans.append(code[i][:-1].strip())
        else:
            ans.append(code[i].strip())

fans = dowhileloop(ans)

print('\nThree address code for given do-while loop code is:')
show = PrettyTable()
show.field_names = ['Serial No','Three address code']
for i in range(len(fans)):
    show.add_row([i+1,fans[i]])

print(show)



Three address code for given do-while loop code is:
+-----------+--------------------+
| Serial No | Three address code |
+-----------+--------------------+
|     1     |     a = b*c+d;     |
|     2     |      b = t-7;      |
|     3     |  if (a<b) goto 5   |
|     4     |       goto 6       |
|     5     |  if (b>c) goto 1   |
|     6     |        END         |
+-----------+--------------------+


In [ ]:
from prettytable import PrettyTable

def display_three_address_code(three_address_code):
    print('\nThe Three Address Code Generated is:')
    table = PrettyTable()
    table.align = "l"
    table.field_names = ['Index','Code']
    for i, code_line in enumerate(three_address_code):
        table.add_row([100 + i, code_line])
    print(table)

# Three Address Code Generated
three_address_code = [
    "t1 = b+c",
    "t2 = t1",
    "a = t2",
    "t3 = t-7",
    "t4 = t3",
    "b = t4",
    "t5 = x+z",
    "t6 = t5",
    "c = t6",
    "if a<b goto 111",
    "goto",
    "if c<b goto 100",
    "goto",
    "End"
]

display_three_address_code(three_address_code)



The Three Address Code Generated is:
+-------+-----------------+
| Index | Code            |
+-------+-----------------+
| 100   | t1 = b+c        |
| 101   | t2 = t1         |
| 102   | a = t2          |
| 103   | t3 = t-7        |
| 104   | t4 = t3         |
| 105   | b = t4          |
| 106   | t5 = x+z        |
| 107   | t6 = t5         |
| 108   | c = t6          |
| 109   | if a<b goto 111 |
| 110   | goto            |
| 111   | if c<b goto 100 |
| 112   | goto            |
| 113   | End             |
+-------+-----------------+
